In [1]:
import pandas as pd
import datetime
import numpy as np
import glob
import string
import re
import gc
import os
gc.collect()

0

In [2]:
# Before
# 2018-01-15 (2nd week) to 2018-05-15 (both dates included)
# 85.7% phone orders found in the PressOne inbound calls
# Use phoen call start time to determin because it can be happened during a call (before the end of the call)


# 15273/32591 46.86% of the total buyer are found with a call in the PressOne dataset
# Jan 15 to Jun 24

In [3]:
today_str=str(datetime.datetime.now().date())
writer_folder="/home/jian/Projects/Saatva/Call_Analysis/output/"+today_str+"/"
try:
    os.stat(writer_folder)
except:
    os.mkdir(writer_folder)

# Transaction Data

In [4]:
klipfolio_data=pd.read_excel("/home/jian/Projects/Saatva/Call_Analysis/data/Transaction_with_phone_2018_to_May.xlsx",sheetname="Sales_By_Brand",dtype=str)
klipfolio_data=klipfolio_data.rename(columns={"Zip_Code":"zip_cd"})
klipfolio_data['Date']=klipfolio_data['Date'].apply(lambda x: datetime.datetime.strptime(x.split(" ")[0],"%Y-%m-%d").date())
klipfolio_data['Sales']=klipfolio_data['Sales'].astype(float)
klipfolio_data['Phone_1']=klipfolio_data['Phone_1'].apply(lambda x: x.replace("-",""))
klipfolio_data['Phone_2']=klipfolio_data['Phone_2'].apply(lambda x: x.replace("-",""))
klipfolio_data['Phone_1']=klipfolio_data['Phone_1'].apply(lambda x: re.sub('['+string.punctuation+']', '', x))
klipfolio_data['Phone_2']=klipfolio_data['Phone_2'].apply(lambda x: re.sub('['+string.punctuation+']', '', x))
klipfolio_data['Phone_1']=klipfolio_data['Phone_1'].apply(lambda x: x.replace(" ",""))
klipfolio_data['Phone_2']=klipfolio_data['Phone_2'].apply(lambda x: x.replace(" ",""))

del klipfolio_data['CSR']
del klipfolio_data['PressOne_Rep_Name']

In [5]:
klipfolio_data_2_folder="/home/jian/Projects/Saatva/Transaction_Data/2018_weekly_klipfolio_with_Phone/"
klipfolio_data_2_file_list=glob.glob(klipfolio_data_2_folder+"*.csv")
for file in klipfolio_data_2_file_list:

    df=pd.read_csv(file,dtype=str)
    df=df[['Order ID','Date','Time','Shipping Zip','Product Revenue','Order Type',
                                   'Phone 1','Phone 2','PressOne Rep Name','Brand']]

    df=df.rename(columns={"Order ID":"Order_id","Shipping Zip":"zip_cd","Product Revenue":"Sales",
                                                     "Order Type":"Source","Phone 1":"Phone_1","Phone 2":"Phone_2",
                                                     "PressOne Rep Name":"PressOne_Rep_Name"})
    
    df=df.fillna("nan")
    df=df[df['Order_id']!="nan"]
    df['Date']=df['Date'].apply(lambda x: x.replace("-April-","-Apr-").replace("-March-","-Mar-").replace("-June-","Jun"))
    df['Date']=np.where(len(df['Date'])<df['Date'].apply(lambda x: len(x)).max(),"0"+df['Date'],df['Date'])
    df['Date']=df['Date'].apply(lambda x: datetime.datetime.strptime(x,"%d-%b-%Y").date())
    df['Sales']=df['Sales'].astype(float)
    df['Phone_1']=df['Phone_1'].apply(lambda x: x.replace("-",""))
    df['Phone_2']=df['Phone_2'].apply(lambda x: x.replace("-",""))
    df['Phone_1']=df['Phone_1'].apply(lambda x: re.sub('['+string.punctuation+']', '', x))
    df['Phone_2']=df['Phone_2'].apply(lambda x: re.sub('['+string.punctuation+']', '', x))
    df['Phone_1']=df['Phone_1'].apply(lambda x: x.replace(" ",""))
    df['Phone_2']=df['Phone_2'].apply(lambda x: x.replace(" ",""))
    

    klipfolio_data=klipfolio_data[klipfolio_data['Date']<=(df['Date'].min()+datetime.timedelta(days=1))]
    klipfolio_data=klipfolio_data.append(df)
    klipfolio_data=klipfolio_data.drop_duplicates()
    
transaction_data=klipfolio_data[klipfolio_data['Brand']=="Saatva"].reset_index()
del transaction_data['index']
del transaction_data['PressOne_Rep_Name']
transaction_data['Time']=transaction_data['Time'].apply(lambda x: x.zfill(8))
transaction_data=transaction_data.drop_duplicates()
transaction_data=transaction_data.reset_index()
del transaction_data['index']

In [6]:
transaction_data_both=transaction_data.copy()
transaction_data_both['Date_Str']=transaction_data_both['Date'].astype(str)
transaction_data_both=transaction_data_both.drop_duplicates()
transaction_data_both['Date_Time_Str']=transaction_data_both['Date_Str']+" "+transaction_data_both['Time']
transaction_data_both['Date_Time']=transaction_data_both['Date_Time_Str'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S"))
del transaction_data_both['Date_Str']
transaction_data_both=transaction_data_both[(transaction_data_both['Date_Time']<datetime.datetime(2018,6,25)) & (transaction_data_both['Date_Time']>=datetime.datetime(2018,1,1))]
data_both_1=transaction_data_both[['Order_id','Date_Time','Phone_1']]
data_both_1=data_both_1.rename(columns={"Phone_1":"Phone"})

data_both_2=transaction_data_both[['Order_id','Date_Time','Phone_2']]
data_both_2=data_both_2.rename(columns={"Phone_2":"Phone"})

transaction_both_data=data_both_1.append(data_both_2)
transaction_both_data=transaction_both_data.sort_values('Date_Time')

In [7]:
x=transaction_both_data[transaction_both_data['Order_id']=="613826"]
x

,Order_id,Date_Time,Phone
33952,613826,2018-06-20 12:08:45,2109122041
33952,613826,2018-06-20 12:08:45,2103945454


In [8]:
y=transaction_both_data[(transaction_both_data['Phone']=="2109122041")| (transaction_both_data['Phone']=="2103945454")]
y

,Order_id,Date_Time,Phone
28207,600883,2018-04-27 16:18:58,2109122041
35644,612025,2018-06-12 10:49:26,2103945454
33952,613826,2018-06-20 12:08:45,2109122041
33952,613826,2018-06-20 12:08:45,2103945454


In [9]:
transaction_data_both.head(2)

,Brand,Date,Order_id,Phone_1,Phone_2,Sales,Source,Time,zip_cd,Date_Time_Str,Date_Time
0,Saatva,2018-04-19,599158,2066617115,nan,999.0,online,23:59:49,98026,2018-04-19 23:59:49,2018-04-19 23:59:49
1,Saatva,2018-04-19,599157,9738381496,9736344041,1227.0,phone,23:54:54,07457,2018-04-19 23:54:54,2018-04-19 23:54:54


In [10]:
transaction_both_data['defined_buyer_id']=np.nan
id_number=1
df_buyer=pd.DataFrame(columns={'Order_id','Date_Time','Phone'})
for order_id,group in transaction_both_data.groupby(['Order_id']):
    df=group[group['Phone']!='nan']
    common_list=list(set(df['Phone'].tolist()).intersection(df_buyer['Phone'].tolist()))
    if len(common_list)>0:
        buyer_id=df_buyer[df_buyer['Phone'].isin(df['Phone'])]['defined_buyer_id'].unique().tolist()
        if len(buyer_id)>1:
            print("Multi_Buyer_id_Error:",order_id)
            df_buyer_re_allocate=df_buyer[df_buyer['defined_buyer_id'].isin(buyer_id)]
            df_buyer=df_buyer[~df_buyer['defined_buyer_id'].isin(buyer_id)]
            df=df_buyer_re_allocate.append(group)
            df=df[df['Phone']!='nan']
            df['defined_buyer_id']=buyer_id[0]
            
        else:    
            df['defined_buyer_id']=buyer_id[0]
    
    else:
        df['defined_buyer_id']="buyer_"+str(id_number)
        id_number=id_number+1
        
    
    df_buyer=df_buyer.append(df)
    
    if id_number% 3000==30:
        print(id_number,datetime.datetime.now())


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


30 2018-07-05 12:24:15.382507


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


3030 2018-07-05 12:25:07.095705
6030 2018-07-05 12:25:59.365007
9030 2018-07-05 12:26:52.401744
12030 2018-07-05 12:27:46.488773
15030 2018-07-05 12:28:44.937416
18030 2018-07-05 12:29:44.533262
21030 2018-07-05 12:30:45.105256
24030 2018-07-05 12:31:46.873015
27030 2018-07-05 12:32:51.431435
30030 2018-07-05 12:33:59.320311
33030 2018-07-05 12:35:09.909501
36030 2018-07-05 12:36:23.543231
Multi_Buyer_id_Error: 613826


In [11]:
# Check
test=df_buyer[(df_buyer['Phone']=="2109122041")| (df_buyer['Phone']=="2103945454")]
test

,Date_Time,Order_id,Phone,defined_buyer_id
28207,2018-04-27 16:18:58,600883,2109122041,buyer_25982
35644,2018-06-12 10:49:26,612025,2103945454,buyer_25982
33952,2018-06-20 12:08:45,613826,2109122041,buyer_25982
33952,2018-06-20 12:08:45,613826,2103945454,buyer_25982


In [12]:
df_buyer=df_buyer.sort_values(['Date_Time','defined_buyer_id'],ascending=[True,True])
df_buyer=df_buyer.reset_index()
del df_buyer['index']

df_order_buyer=df_buyer[['Date_Time','Order_id','defined_buyer_id']].drop_duplicates().reset_index()
del df_order_buyer['index']

In [13]:
df_order_buyer_seq=pd.DataFrame()
for defined_buyer_id,group in df_order_buyer.groupby(['defined_buyer_id']):
    group['Buy_Seq']=[x for x in range(1,(len(group)+1))]
    df_order_buyer_seq=df_order_buyer_seq.append(group)
print(datetime.datetime.now())

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


2018-07-05 12:49:41.174325


In [14]:
transaction_data=pd.merge(transaction_data_both,df_order_buyer_seq,on=['Order_id','Date_Time'],how="outer")
transaction_data=transaction_data.sort_values(['Date_Time'])

In [15]:
# For multiple purchases, the Phone_2 issue not cleaned yet!!! (2 records for the same order, 1 with 2 phones and 1 with 1 phone)
transaction_data_1st_Only=transaction_data[transaction_data['Buy_Seq']==1]
transaction_data_1st_Only=transaction_data_1st_Only.sort_values(['Date_Time','Order_id','defined_buyer_id','Phone_1','Phone_2'])
transaction_data_1st_Only=transaction_data_1st_Only.drop_duplicates(['Date_Time','Order_id','defined_buyer_id','Phone_1'])
transaction_data_phone_1st_Only=transaction_data_1st_Only[transaction_data_1st_Only['Source']=="phone"]
transaction_data_online_1st_Only=transaction_data_1st_Only[transaction_data_1st_Only['Source']=="online"]

In [16]:
transaction_data_1st_Only.shape[0]-len(transaction_data_phone_1st_Only['defined_buyer_id'].unique())-len(transaction_data_online_1st_Only['defined_buyer_id'].unique())



0

In [17]:
data_both_1=transaction_data[['Phone_1','defined_buyer_id']]
data_both_1=data_both_1.rename(columns={"Phone_1":"Phone"})

data_both_2=transaction_data[['Phone_2','defined_buyer_id']]
data_both_2=data_both_2.rename(columns={"Phone_2":"Phone"})

match_buyer_phone=data_both_1.append(data_both_2).drop_duplicates()
match_buyer_phone=match_buyer_phone[match_buyer_phone['Phone']!='nan']

In [18]:
match_buyer_phone_list=match_buyer_phone.groupby(['defined_buyer_id'])['Phone'].apply(list).to_frame().reset_index()
match_buyer_phone_list.tail(2)

,defined_buyer_id,Phone
38291,buyer_9998,"[9065537223, 9065532849]"
38292,buyer_9999,[7405456547]


In [19]:
transacted_match_data_writer=pd.ExcelWriter(writer_folder+"transaction_match_data"+today_str+".xlsx",engine="xlsxwriter")
transaction_data.to_excel(transacted_match_data_writer,"transactation_data",index=False)
transaction_data_1st_Only.to_excel(transacted_match_data_writer,"trans_data_1st_purchase",index=False)
match_buyer_phone_list.to_excel(transacted_match_data_writer,"buyer_phone",index=False)
transacted_match_data_writer.save()



In [20]:
'''
Multiple_Buyers=transaction_data[transaction_data['defined_buyer_id'].isin(transaction_data[transaction_data['Buy_Seq']>1]['defined_buyer_id'])]
len(Multiple_Buyers['defined_buyer_id'].unique()) #660
'''



# In total 4 unique orders (also unique buyers) contain the free code, and 2 of the 4 have another phone (4-2 only with the 866 and 888)
# 8665963368,9072541011
# 4157389811,8772727241
# 8662266454
# 8886390639
'''
copy=transaction_data.copy()
copy['AC_1']=copy['Phone_1'].apply(lambda x: str(x)[0:3])
copy['AC_2']=copy['Phone_2'].apply(lambda x: str(x)[0:3])

copy[(copy['AC_1'].isin(['800','888','877','866','855','844'])) | (copy['AC_2'].isin(['800','888','877','866','855','844']))]
'''
'''
digit_11=match_buyer_phone[match_buyer_phone['Phone'].apply(lambda x: len(x))==11]
digit_11['1st_digit']=digit_11['Phone'].apply(lambda x: x[0])
digit_11['last_10']=digit_11['Phone'].apply(lambda x: x[1:])
digit_11.groupby(['1st_digit'])['defined_buyer_id'].count().to_frame().reset_index()
# 24 out of the 94 are found if remove the leading 1 for the 11-digits phones
# No need to clean the leading 1, because of low counts, low pctg fount, and unsure of the country code
'''
'''
transaction_data_online=transaction_data[transaction_data['Source']=="online"]
transaction_data_online['Date_Str']=transaction_data_online['Date'].astype(str)
transaction_data_online['Date_Time']=transaction_data_online['Date_Str']+" "+transaction_data_online['Time']
transaction_data_online['Time']=transaction_data_online['Date_Time'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S"))
del transaction_data_online['Date_Str']
transaction_data_online=transaction_data_online[(transaction_data_online['Time']<=datetime.datetime(2018,6,10)) & (transaction_data_phone['Time']>=datetime.datetime(2018,1,15))]
data_online_1=transaction_data_online[['Order_id','Date_Time','Phone_1']]
data_online_2=transaction_data_online[['Order_id','Date_Time','Phone_2']]
'''
'''
transaction_data_phone=transaction_data[transaction_data['Source']=="phone"]
transaction_data_phone['Date_Str']=transaction_data_phone['Date'].astype(str)
transaction_data_phone['Date_Time']=transaction_data_phone['Date_Str']+" "+transaction_data_phone['Time']
transaction_data_phone['Time']=transaction_data_phone['Date_Time'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S"))
del transaction_data_phone['Date_Str']
transaction_data_phone=transaction_data_phone[(transaction_data_phone['Time']<=datetime.datetime(2018,6,10)) & (transaction_data_phone['Time']>=datetime.datetime(2018,1,15))]
data_phone_1=transaction_data_phone[['Order_id','Date_Time','Phone_1']]
data_phone_1=data_phone_1.rename(columns={"Phone_1":"Phone"})
data_phone_2=transaction_data_phone[['Order_id','Date_Time','Phone_2']]
data_phone_2=data_phone_2.rename(columns={"Phone_2":"Phone"})

transaction_data_phone=data_phone_1.append(data_phone_2)
'''

'\ntransaction_data_phone=transaction_data[transaction_data[\'Source\']=="phone"]\ntransaction_data_phone[\'Date_Str\']=transaction_data_phone[\'Date\'].astype(str)\ntransaction_data_phone[\'Date_Time\']=transaction_data_phone[\'Date_Str\']+" "+transaction_data_phone[\'Time\']\ntransaction_data_phone[\'Time\']=transaction_data_phone[\'Date_Time\'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S"))\ndel transaction_data_phone[\'Date_Str\']\ntransaction_data_phone=transaction_data_phone[(transaction_data_phone[\'Time\']<=datetime.datetime(2018,6,10)) & (transaction_data_phone[\'Time\']>=datetime.datetime(2018,1,15))]\ndata_phone_1=transaction_data_phone[[\'Order_id\',\'Date_Time\',\'Phone_1\']]\ndata_phone_1=data_phone_1.rename(columns={"Phone_1":"Phone"})\ndata_phone_2=transaction_data_phone[[\'Order_id\',\'Date_Time\',\'Phone_2\']]\ndata_phone_2=data_phone_2.rename(columns={"Phone_2":"Phone"})\n\ntransaction_data_phone=data_phone_1.append(data_phone_2)\n'

# Buyer_DMA

In [21]:
buyer_zip=transaction_data[['defined_buyer_id','zip_cd']].drop_duplicates()
zip_DMA=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",dtype=str,skiprows=1)
zip_DMA=zip_DMA.iloc[:,[0,2]]
zip_DMA.columns=['zip_cd',"DMA"]
zip_DMA=zip_DMA.drop_duplicates(['zip_cd'])
buyer_zip_DMA=pd.merge(buyer_zip,zip_DMA,on='zip_cd',how="left")
buyer_zip_DMA=buyer_zip_DMA.fillna("non_US")
# 4 zips in 5 digits are not in DMA 80847,02098,49165,22512

# Phone Data

In [22]:
Press_One_folder="/home/jian/Projects/Saatva/Call_Analysis/data/2018_Call_Data_with_Dup/"
call_file_list=glob.glob(Press_One_folder+"*.csv")
call_data=pd.DataFrame()
for file in call_file_list:
    df=pd.read_csv(file,dtype=str)
    call_data=call_data.append(df)
call_data=call_data.drop_duplicates()

call_data.reset_index(inplace=True)
del call_data['index']
call_data=call_data[['S','Started (Dist) (Display)','Talk Time (Display)','Talk Time (Seconds Value)','Ring (Dist) (Seconds Value)','Dev (Clg)','TelNo (Clg)','Ans','Type']]
call_data.columns=['Status','Start_Time_Str','Talk_Time_Str','Talk_Time_Sec','Ring_Time_Sec','Dev_Clg','TelNo_Clg','Ans','Type']
call_data=call_data[~pd.isnull(call_data['Status'])]
call_data=call_data.reset_index()
del call_data['index']
gc.collect()

51500

In [23]:
call_data=call_data[call_data['Type']=="Ext/In"] # imbound calls only

call_data['Call_in_Phone']=np.where(call_data['Dev_Clg']=="[Trunk]",
                                   call_data['TelNo_Clg'],call_data['Dev_Clg'])
call_data=call_data[(call_data['Call_in_Phone']!="[No CallerID]") & (call_data['Call_in_Phone']!="[Conference]") & (call_data['Call_in_Phone']!="Anonymous") & (call_data['Call_in_Phone']!="0000000000")]
call_data['Call_in_Phone']=call_data['Call_in_Phone'].apply(lambda x:x.split("x")[0])

call_data['Start_Time']=call_data['Start_Time_Str'].apply(lambda x: datetime.datetime.strptime(x,'%m/%d/%Y %H:%M:%S'))
call_data['Talk_Time_Delta']=call_data['Talk_Time_Sec'].astype(int).apply(lambda x:datetime.timedelta(seconds=x))
call_data['Ring_Time_Delta']=call_data['Ring_Time_Sec'].astype(int).apply(lambda x:datetime.timedelta(seconds=x))

call_data['End_Time']=call_data['Start_Time']+call_data['Talk_Time_Delta']+call_data['Ring_Time_Delta']
call_data['Area_Code']=call_data['Call_in_Phone'].apply(lambda x: x[0:3])
call_data=call_data.sort_values(['Call_in_Phone','Start_Time','End_Time'])


free_numbers_area=['800','888','877','866','855','844']
call_data=call_data[~call_data['Area_Code'].isin(free_numbers_area)]
call_data=call_data.reset_index()
call_data['Phone']=call_data['Call_in_Phone']

call_data=call_data[(call_data['Start_Time']<datetime.datetime(2018,6,25)) & (call_data['Start_Time']>=datetime.datetime(2018,1,1))]
call_data=call_data.sort_values(['Call_in_Phone','Start_Time'])
call_data=call_data.reset_index()
del call_data['index']

In [24]:
call_data=pd.merge(call_data,match_buyer_phone,on="Phone",how="left")
call_data=call_data.sort_values(['Call_in_Phone','Start_Time'],ascending=[True,True])

call_data_transacted=call_data[~pd.isnull(call_data['defined_buyer_id'])]
call_data_transacted=call_data_transacted.sort_values(['Call_in_Phone','Start_Time'],ascending=[True,True])

call_data_no_buy=call_data[pd.isnull(call_data['defined_buyer_id'])]
call_data_no_buy=call_data_no_buy.sort_values(['Call_in_Phone','Start_Time'],ascending=[True,True])

In [25]:
call_data.shape

(275771, 18)

In [26]:
call_data_transacted.shape

(89814, 18)

In [27]:
call_data_no_buy.shape

(185957, 18)

In [28]:
# len(set(digit_11[digit_11['1st_digit']=="1"]['last_10'].unique().tolist()).intersection(call_data['Phone'].unique().tolist()))

# Non-Buyers

In [ ]:
# Reason to use 30 seconds: e.g. June, Phone "2025094332"
# Part 1: non-transacted phone calls
gc.collect()
call_data_group_only_no_buy=call_data_no_buy[['Call_in_Phone','Start_Time','End_Time','Talk_Time_Delta','Ring_Time_Delta']]
non_transacted_phone_calls=pd.DataFrame()

k=1
for phone,group in call_data_group_only_no_buy.groupby(['Call_in_Phone']):

    df_drop_transfer=group.reset_index()
    del df_drop_transfer['index']
    df_drop_transfer=df_drop_transfer.sort_values(['Start_Time']) # Order!
    df_drop_transfer=df_drop_transfer.reset_index()
    del df_drop_transfer['index']
    df_drop_transfer['call_seq']=np.nan
    df_drop_transfer['call_seq'][0]=1
    seq=1

    for i in range(1,len(df_drop_transfer)):
        if (df_drop_transfer['Start_Time'][i]-df_drop_transfer['End_Time'][i-1]).seconds<=30:
            df_drop_transfer['call_seq'][i]=seq
        else:
            seq=seq+1
            df_drop_transfer['call_seq'][i]=seq

    non_transacted_phone_calls=non_transacted_phone_calls.append(df_drop_transfer)
    k=k+1
    if k%10000==100: # <10 minutes of running per 10,000 loops
        print(k,datetime.datetime.now())

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


100 2018-07-05 10:48:56.977137


In [ ]:
gc.collect()
i=0
unique_non_transacted_phone_calls=pd.DataFrame()
for (phone,seq),group in non_transacted_phone_calls.groupby(['Call_in_Phone','call_seq']):
    df_unique_phone=group.reset_index()
    del df_unique_phone['index']
    
    len_df=len(df_unique_phone)
    start_time=df_unique_phone['Start_Time'][0]
    end_time=df_unique_phone['End_Time'][len_df-1]
    duration_sec=(end_time-start_time).seconds
    talk_sec=df_unique_phone['Talk_Time_Delta'].sum()
    ring_sec=df_unique_phone['Ring_Time_Delta'].sum()
    
    df=pd.DataFrame({"Phone_Num":phone,"Call_Seq":seq,"Start_time":start_time,"End_time":end_time,"Call_duration_Sec":duration_sec,
                    "Talk_length_Sec":talk_sec,"Ring_length_Sec":ring_sec},index=[i])
    unique_non_transacted_phone_calls=unique_non_transacted_phone_calls.append(df)
    i=i+1
    if i%10000==100:
        print(i,datetime.datetime.now())

In [ ]:
unique_non_transacted_phone_calls_Exclusion=unique_non_transacted_phone_calls[unique_non_transacted_phone_calls['Phone_Num'].isin(unique_non_transacted_phone_calls[unique_non_transacted_phone_calls['Call_Seq']>=100]['Phone_Num'])]
unique_non_transacted_phone_calls=unique_non_transacted_phone_calls[~unique_non_transacted_phone_calls['Phone_Num'].isin(unique_non_transacted_phone_calls_Exclusion['Phone_Num'])]




# Buyers

In [ ]:
transaction_time_1st_Only=transaction_data_1st_Only[['defined_buyer_id','Date_Time','Source']]
transaction_time_1st_Only=transaction_time_1st_Only.rename(columns={"Date_Time":"Trans_1st_Time"})
transaction_time_1st_Only=transaction_time_1st_Only.reset_index()
del transaction_time_1st_Only['index']
transaction_time_1st_Only['Trans_1st_Time'][0]
transaction_time_1st_Only['Trans_1st_Time_plust_1_hour']=transaction_time_1st_Only['Trans_1st_Time'].apply(lambda x: x+datetime.timedelta(hours=1))

In [ ]:
# Part 2: Transacted phone calls
gc.collect()
call_data_group_buyers=call_data_transacted[['defined_buyer_id','Start_Time','End_Time','Talk_Time_Delta','Ring_Time_Delta']]
Transacted_phone_calls=pd.DataFrame()


In [ ]:
call_data_group_buyers.head(2)

In [ ]:
k=1
for buyer_id,group in call_data_group_buyers.groupby(['defined_buyer_id']):

    df_drop_transfer=group.reset_index()
    del df_drop_transfer['index']
    df_drop_transfer=df_drop_transfer.sort_values(['Start_Time']) #Order
    df_drop_transfer=df_drop_transfer.reset_index()
    del df_drop_transfer['index']
    df_drop_transfer['call_seq']=np.nan
    df_drop_transfer['call_seq'][0]=1
    seq=1

    if len(df_drop_transfer)>1:
        for i in range(1,len(df_drop_transfer)):
            if (df_drop_transfer['Start_Time'][i]-df_drop_transfer['End_Time'][i-1]).seconds<=30:
                df_drop_transfer['call_seq'][i]=seq
            else:
                seq=seq+1
                df_drop_transfer['call_seq'][i]=seq

    Transacted_phone_calls=Transacted_phone_calls.append(df_drop_transfer)
    k=k+1
    if k%1500==100:
        print(k,datetime.datetime.now())


In [ ]:
gc.collect()
i=0
unique_Transacted_phone_calls=pd.DataFrame()
for (buyer_id,seq),group in Transacted_phone_calls.groupby(['defined_buyer_id','call_seq']):
    df_unique_phone=group.reset_index()
    del df_unique_phone['index']
    len_df=len(df_unique_phone)
    start_time=df_unique_phone['Start_Time'][0]
    end_time=df_unique_phone['End_Time'][len_df-1]
    duration_sec=(end_time-start_time).seconds
    talk_sec=df_unique_phone['Talk_Time_Delta'].sum()
    ring_sec=df_unique_phone['Ring_Time_Delta'].sum()
    
    df=pd.DataFrame({"defined_buyer_id":buyer_id,"Call_Seq":seq,"Start_time":start_time,"End_time":end_time,"Call_duration_Sec":duration_sec,
                    "Talk_length_Sec":talk_sec,"Ring_length_Sec":ring_sec},index=[i])
    unique_Transacted_phone_calls=unique_Transacted_phone_calls.append(df)
    i=i+1
    if i%3700==100:
        print(i,datetime.datetime.now())

In [ ]:
unique_Transacted_phone_calls.head(2)

In [ ]:
unique_Transacted_phone_calls_Before=pd.merge(unique_Transacted_phone_calls,transaction_time_1st_Only,on="defined_buyer_id",how="left")

# Online_Order: call Start time only before Transaction time
# Phone_Order: call End time only before Transaction time + 1 Hour


unique_Transacted_calls_Before_phone=unique_Transacted_phone_calls_Before[unique_Transacted_phone_calls_Before['Source']=="phone"]
unique_Transacted_calls_Before_phone=unique_Transacted_calls_Before_phone[unique_Transacted_calls_Before_phone['End_time']<=unique_Transacted_calls_Before_phone['Trans_1st_Time_plust_1_hour']]

unique_Transacted_calls_Before_online=unique_Transacted_phone_calls_Before[unique_Transacted_phone_calls_Before['Source']=="online"]
unique_Transacted_calls_Before_online=unique_Transacted_calls_Before_online[unique_Transacted_calls_Before_online['Start_time']<=unique_Transacted_calls_Before_online['Trans_1st_Time']]

unique_Transacted_phone_calls_matched=unique_Transacted_calls_Before_phone.append(unique_Transacted_calls_Before_online)
unique_Transacted_phone_calls_matched=unique_Transacted_phone_calls_matched.sort_values(['Start_time'])

In [ ]:
unique_Transacted_phone_calls_matched['Source'].unique()

In [ ]:
unique_Transacted_phone_calls_matched['Start_time'].max()

In [ ]:
unique_Transacted_phone_calls_matched['Start_time'].min()

In [ ]:
len(unique_Transacted_phone_calls_matched[unique_Transacted_phone_calls_matched['Source']=='phone']['defined_buyer_id'].unique())

In [ ]:
writer=pd.ExcelWriter(writer_folder+"transacted_and_non_transacted_call_data"+today_str+".xlsx",engine="xlsxwriter")
call_data.to_excel(writer,"call_data",index=False)
non_transacted_phone_calls.to_excel(writer,'non_transcted_call_all',index=False)
unique_non_transacted_phone_calls.to_excel(writer,'non_transcted_unique',index=False)
Transacted_phone_calls.to_excel(writer,'Transcted_call_all',index=False)
unique_Transacted_phone_calls_matched.to_excel(writer,'Transcted_unique',index=False)
writer.save()

In [37]:


unique_Transacted_phone_calls_matched.head(2)

,Call_Seq,Call_duration_Sec,End_time,Ring_length_Sec,Start_time,Talk_length_Sec,defined_buyer_id,Trans_1st_Time,Source,Trans_1st_Time_plust_1_hour
0,1,844,2018-01-01 02:13:39,2.314814814814815e-05,2018-01-01 01:59:35,0.009733796296296296,buyer_6,2018-01-01 02:15:51,phone,2018-01-01 03:15:51
1,1,1199,2018-01-01 09:16:14,2.314814814814815e-05,2018-01-01 08:56:15,0.01384259259259259,buyer_29707,2018-05-15 16:54:31,phone,2018-05-15 17:54:31


In [93]:
transaction_data_1st_Only['Date_Time'].apply(lambda x:type(x)).unique()

array([<class 'pandas._libs.tslib.Timestamp'>], dtype=object)

In [96]:
df_July_04=pd.read_excel("/home/jian/Projects/Saatva/Call_Analysis/output/2018-07-04/transacted_and_non_transacted_call_data2018-07-04.xlsx",sheetname=None,dtype=str)

non_transacted_phone_calls=df_July_04['non_transcted_call_all']
unique_non_transacted_phone_calls=df_July_04['non_transcted_unique']
Transacted_phone_calls=df_July_04['Transcted_call_all']
unique_Transacted_phone_calls_matched=df_July_04['Transcted_unique']



In [97]:
non_transacted_phone_calls['Start_Time']=non_transacted_phone_calls['Start_Time'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S"))
unique_non_transacted_phone_calls['Start_time']=unique_non_transacted_phone_calls['Start_time'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S"))
Transacted_phone_calls['Start_Time']=Transacted_phone_calls['Start_Time'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S"))
unique_Transacted_phone_calls_matched['Start_time']=unique_Transacted_phone_calls_matched['Start_time'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S"))
unique_Transacted_phone_calls_matched['Call_Seq']=unique_Transacted_phone_calls_matched['Call_Seq'].astype(int)

In [98]:
Transacted_Call_Freq_by_buyers=unique_Transacted_phone_calls_matched.groupby(['defined_buyer_id'])['Start_time'].count().to_frame().reset_index()
Transacted_Call_Freq_by_buyers=Transacted_Call_Freq_by_buyers.rename(columns={"Start_time":"Call_Times"})

Transacted_Call_Freq_by_buyers_1_to_4=Transacted_Call_Freq_by_buyers[Transacted_Call_Freq_by_buyers['Call_Times']<=4]
Transacted_Call_Freq_by_buyers_5_plus=Transacted_Call_Freq_by_buyers[Transacted_Call_Freq_by_buyers['Call_Times']>4]


# Df_1 Call Summary

In [99]:
def agg_by_freq_seg(df,df_5_plus):
    df_1=df.groupby(['Call_Times'])['defined_buyer_id'].apply(list).to_frame().reset_index()
    df_1=df_1.rename(columns={"defined_buyer_id":"buyer_list"})

    df_2=df.groupby(['Call_Times'])['defined_buyer_id'].count().to_frame().reset_index()
    df_2=df_2.rename(columns={"defined_buyer_id":"buyer_count"})

    df_result=pd.merge(df_1,df_2,on="Call_Times",how="outer")
    
    df_5_plus_app=pd.DataFrame({"Call_Times":"5+","buyer_list":[df_5_plus['defined_buyer_id'].unique().tolist()],"buyer_count":len(df_5_plus['defined_buyer_id'].unique().tolist())},index=[4])
    df_result=df_result.append(df_5_plus_app)
    
    return df_result

base_df_transacted=agg_by_freq_seg(Transacted_Call_Freq_by_buyers_1_to_4,Transacted_Call_Freq_by_buyers_5_plus)

In [100]:

# 1. Transacted
Transaced_df_summary=base_df_transacted.copy()
Transaced_df_summary['Total_Buyer']=np.nan
Transaced_df_summary['Phone_Buyer']=np.nan
Transaced_df_summary['Online_Buyer']=np.nan

Transaced_df_summary['Total_Trans']=np.nan
Transaced_df_summary['Phone_Trans']=np.nan
Transaced_df_summary['Online_Trans']=np.nan

Transaced_df_summary['Total_Transacted_Calls']=np.nan

# Use 1st Time Transaction only : transaction_data_1st_Only
# Because use the 1st time transaction, the total equals to the sum of both sources
for i in range(5):
    buyer_list=Transaced_df_summary['buyer_list'][i]
    df_trans=transaction_data[transaction_data['defined_buyer_id'].isin(buyer_list)]
    type_of_order_order=df_trans.groupby(['Source'])['Order_id'].count().to_frame().reset_index()
    
    phone_trans_order=type_of_order_order[type_of_order_order['Source']=="phone"]['Order_id'].unique()[0]
    online_trans_order=type_of_order_order[type_of_order_order['Source']=="online"]['Order_id'].unique()[0]
    both_trans_order=len(df_trans['Order_id'].unique())
    
    df=transaction_data_1st_Only[transaction_data_1st_Only['defined_buyer_id'].isin(buyer_list)]
    type_of_order_buyer=df.groupby(['Source'])['defined_buyer_id'].apply(lambda x: len(x.unique())).to_frame().reset_index()
    phone_trans_buyer=type_of_order_buyer[type_of_order_buyer['Source']=="phone"]['defined_buyer_id'].unique()[0]
    online_trans_buyer=type_of_order_buyer[type_of_order_buyer['Source']=="online"]['defined_buyer_id'].unique()[0]
    both_trans_buyer=len(df['Order_id'].unique())

    
    Transaced_df_summary['Phone_Trans'][i]=phone_trans_order
    Transaced_df_summary['Online_Trans'][i]=online_trans_order
    Transaced_df_summary['Total_Trans'][i]=both_trans_order
    
    Transaced_df_summary['Phone_Buyer'][i]=phone_trans_buyer
    Transaced_df_summary['Online_Buyer'][i]=online_trans_buyer
    Transaced_df_summary['Total_Buyer'][i]=both_trans_buyer
    
    df_total_call=unique_Transacted_phone_calls_matched[unique_Transacted_phone_calls_matched['defined_buyer_id'].isin(buyer_list)]
    total_calls=len(df_total_call)
    Transaced_df_summary['Total_Transacted_Calls'][i]=total_calls
    
Transaced_df_summary=Transaced_df_summary[['Call_Times','buyer_count','buyer_list','Total_Transacted_Calls','Phone_Trans','Online_Trans','Total_Trans',
                                          'Phone_Buyer','Online_Buyer','Total_Buyer']]

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See th

In [101]:
#2. Non Buyers
unique_non_transacted_phone_calls.head(2)
Non__Transacted_df_summary_count=unique_non_transacted_phone_calls.groupby(['Phone_Num'])['Start_time'].count().to_frame().reset_index()
# exclusions_non_transactions=Non__Transacted_df_summary_count[Non__Transacted_df_summary_count['Start_time']>=100]
# Non__Transacted_df_summary_count=Non__Transacted_df_summary_count[Non__Transacted_df_summary_count['Start_time']<100]


Non__Transacted_df_summary_list=Non__Transacted_df_summary_count.groupby(['Start_time'])['Phone_Num'].apply(list).to_frame().reset_index()
Non__Transacted_df_summary_list=Non__Transacted_df_summary_list.rename(columns={"Start_time":"Call_Times","Phone_Num":"Non_Transacted_Phones"})

Non__Transacted_df_summary_list_1_4=Non__Transacted_df_summary_list[Non__Transacted_df_summary_list['Call_Times']<=4]

Non__Transacted_df_summary_list_5_Plus_list_Phone=Non__Transacted_df_summary_count[Non__Transacted_df_summary_count['Start_time']>=5]['Phone_Num'].unique().tolist()
Non__Transacted_df_summary=Non__Transacted_df_summary_list_1_4.append(pd.DataFrame({"Call_Times":"5+","Non_Transacted_Phones":[Non__Transacted_df_summary_list_5_Plus_list_Phone]}))
Non__Transacted_df_summary['Non_Transacted_Calls']=np.nan

Non__Transacted_df_summary=Non__Transacted_df_summary.reset_index()
del Non__Transacted_df_summary['index']
for i in range(len(Non__Transacted_df_summary)):
    phone_list=Non__Transacted_df_summary['Non_Transacted_Phones'][i]
    df=unique_non_transacted_phone_calls[unique_non_transacted_phone_calls['Phone_Num'].isin(phone_list)]
    counts=len(df)
    Non__Transacted_df_summary['Non_Transacted_Calls'][i]=counts
Non__Transacted_df_summary['Unique_Non_Trancted_Call_Count']=Non__Transacted_df_summary['Non_Transacted_Phones'].apply(len)
    


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [102]:
df_1_output=pd.merge(Transaced_df_summary,Non__Transacted_df_summary,on='Call_Times',how="left")
df_1_output['Total_Overall_Calls']=df_1_output['Non_Transacted_Calls']+df_1_output['Total_Transacted_Calls']
df_1_writer=pd.ExcelWriter(writer_folder+"Output_1_Summary_"+today_str+".xlsx",engine="xlsxwriter")
df_1_output.to_excel(df_1_writer,"Output_1_Summary.csv",index=False)
df_1_writer.save()


# Df_2 From Call to Trans Time length

In [103]:
# Transacted data only
call_diff_time_writer=pd.ExcelWriter(writer_folder+"Output_2_Five_Group_Call_to_Transactions"+today_str+".xlsx",engine="xlsxwriter")
Call_to_Trans_Length=base_df_transacted.copy()
Call_to_Trans_Length['Day_Length']=np.nan

Call_to_Trans_Length_call=pd.DataFrame()
Call_to_Trans_Length_online=pd.DataFrame()
for i in range(len(Call_to_Trans_Length)):
    buyer_list=Call_to_Trans_Length['buyer_list'][i]
    df_trans=transaction_data_1st_Only[transaction_data_1st_Only['defined_buyer_id'].isin(buyer_list)]
    df_trans['Trans_1st_Time_plust_1_hour']=df_trans['Date_Time'].apply(lambda x: x+datetime.timedelta(hours=1))
    
    df_trans_time=df_trans[['defined_buyer_id','Trans_1st_Time_plust_1_hour','Source']]
    df_trans_time=df_trans_time.rename(columns={"Trans_1st_Time_plust_1_hour":"Trans_Time_plus_1"})
    
    df_call=unique_Transacted_phone_calls_matched[unique_Transacted_phone_calls_matched['defined_buyer_id'].isin(buyer_list)]
    df_call=df_call[df_call['Call_Seq']==1]
    df_call_time=df_call[['defined_buyer_id','Start_time']]
    df_call_time=df_call_time.rename(columns={"Start_time":"Call_First_Time"})
    if len(df_trans_time)!=len(df_trans_time):
        print(i)
    df_diff=pd.merge(df_call_time,df_trans_time,on="defined_buyer_id",how='outer')
    df_diff['Buyer_Group']='Call_Times_'+str(Call_to_Trans_Length['Call_Times'][i])
    
    
    
    df_diff_online=df_diff[df_diff['Source']=="online"]
    df_diff_phone=df_diff[df_diff['Source']=="phone"]

    df_diff_online['Diff_Time']=df_diff_online['Trans_Time_plus_1']-df_diff_online['Call_First_Time']
    df_diff_online['Day_Length']=df_diff_online['Diff_Time'].apply(lambda x: x.days) #Use +1 Hour
    df_diff_online['Source']='online'
    
    df_diff_phone['Diff_Time']=df_diff_phone['Trans_Time_plus_1']-df_diff_phone['Call_First_Time']
    df_diff_phone['Day_Length']=df_diff_phone['Diff_Time'].apply(lambda x: x.days) #Use +1 Hour
    df_diff_phone['Source']='phone'
    
    df_diff=df_diff_online.append(df_diff_phone)
    df_diff.to_excel(call_diff_time_writer,"group_freq_"+str(i+1),index=False)
    
    Call_to_Trans_Length['Day_Length'][i]=df_diff['Day_Length'].mean()
    
    buyer_list_call=df_diff_phone['defined_buyer_id'].unique().tolist()
    buyer_list_online=df_diff_online['defined_buyer_id'].unique().tolist()
    
    call_app=pd.DataFrame({"Call_Times":str(i+1),"buyer_count":len(buyer_list_call),\
                          "buyer_list":[buyer_list_call],"Day_Length":df_diff_phone['Day_Length'].mean()},index=[i])
    online_app=pd.DataFrame({"Call_Times":str(i+1),"buyer_count":len(buyer_list_online),\
                          "buyer_list":[buyer_list_online],"Day_Length":df_diff_online['Day_Length'].mean()},index=[i])
    Call_to_Trans_Length_call=Call_to_Trans_Length_call.append(call_app)
    Call_to_Trans_Length_online=Call_to_Trans_Length_online.append(online_app)
    
    
Call_to_Trans_Length.to_excel(call_diff_time_writer,"avg_length_overall",index=False)

Call_to_Trans_Length_call['Call_Times']=Call_to_Trans_Length_call['Call_Times'].apply(lambda x: x.replace("5","5+"))
Call_to_Trans_Length_online['Call_Times']=Call_to_Trans_Length_online['Call_Times'].apply(lambda x: x.replace("5","5+"))

Call_to_Trans_Length_call.to_excel(call_diff_time_writer,"avg_length_call",index=False)
Call_to_Trans_Length_online.to_excel(call_diff_time_writer,"avg_length_online",index=False)

call_diff_time_writer.save()

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pyda

# Df_3 Weekday Call Volumes

In [104]:
weekday_call_df_non_trans=unique_non_transacted_phone_calls[['Phone_Num','Start_time','Call_Seq']]
weekday_call_df_non_trans.columns=["Phone|Buyer","Start_time_Call",'Call_Seq']
weekday_call_df_non_trans['weekday']=weekday_call_df_non_trans['Start_time_Call'].apply(lambda x: str(x.weekday()))
weekday_call_df_non_trans['Day_of_Week']=weekday_call_df_non_trans['weekday'].apply(lambda x: x.replace("0","Monday").\
                                                                                    replace("1","Tuesday").replace("2","Wednesday").\
                                                                                   replace("3","Thursday").replace("4","Friday").\
                                                                                   replace("5","Saturday").replace("6","Sunday"))
                                                                   
                                                                   
weekday_call_df_transacted=unique_Transacted_phone_calls_matched[['defined_buyer_id','Start_time','Call_Seq']]
weekday_call_df_transacted.columns=["Phone|Buyer","Start_time_Call",'Call_Seq']
weekday_call_df_transacted['weekday']=weekday_call_df_transacted['Start_time_Call'].apply(lambda x: str(x.weekday()))
weekday_call_df_transacted['Day_of_Week']=weekday_call_df_transacted['weekday'].apply(lambda x: x.replace("0","Monday").\
                                                                                    replace("1","Tuesday").replace("2","Wednesday").\
                                                                                   replace("3","Thursday").replace("4","Friday").\
                                                                                   replace("5","Saturday").replace("6","Sunday"))

weekday_call_df_both=weekday_call_df_non_trans.append(weekday_call_df_transacted)


output_3_both=weekday_call_df_both.groupby(['weekday','Day_of_Week'])['Phone|Buyer'].count().to_frame().reset_index()
output_3_both=output_3_both.rename(columns={"Phone|Buyer":"Call_From_Both"})
output_3_transacted=weekday_call_df_transacted.groupby(['weekday','Day_of_Week'])['Phone|Buyer'].count().to_frame().reset_index()
output_3_transacted=output_3_transacted.rename(columns={"Phone|Buyer":"Call_From_Buyers"})
output_3_non_trans=weekday_call_df_non_trans.groupby(['weekday','Day_of_Week'])['Phone|Buyer'].count().to_frame().reset_index()
output_3_non_trans=output_3_non_trans.rename(columns={"Phone|Buyer":"Call_From_Non_Buyers"})

output_3_weekday_overll=pd.merge(output_3_transacted,output_3_non_trans,on=['weekday',"Day_of_Week"])
output_3_weekday_overll=pd.merge(output_3_weekday_overll,output_3_both,on=['weekday',"Day_of_Week"])



/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

In [105]:
# 3.1 Buyer 
weekday_group_buyer=pd.DataFrame(columns={'Buyer_List',"Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"},index=['1','2','3','4','5+'])
weekday_group_buyer=weekday_group_buyer.reset_index()
weekday_group_buyer=weekday_group_buyer.rename(columns={"index":"Call_Times"})
weekday_group_buyer=weekday_group_buyer[['Call_Times','Buyer_List']+["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]]

for i in range(len(Transaced_df_summary)):
    Buyer_List=Transaced_df_summary['buyer_list'][i]
    df_call=unique_Transacted_phone_calls_matched[unique_Transacted_phone_calls_matched['defined_buyer_id'].isin(Buyer_List)][['defined_buyer_id','Start_time','Call_Seq']]
    df_call.columns=["Phone|Buyer","Start_time_Call",'Call_Seq']
    df_call['weekday']=df_call['Start_time_Call'].apply(lambda x: str(x.weekday()))
    df_call['Day_of_Week']=df_call['weekday'].apply(lambda x: x.replace("0","Monday").\
                                                                                    replace("1","Tuesday").replace("2","Wednesday").\
                                                                                   replace("3","Thursday").replace("4","Friday").\
                                                                                   replace("5","Saturday").replace("6","Sunday"))
    df_group_weekday=df_call.groupby(['Day_of_Week'])['Phone|Buyer'].count().to_frame().reset_index()
    for weekday in ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]:
        weekday_group_buyer[weekday][i]=df_group_weekday[df_group_weekday['Day_of_Week']==weekday]['Phone|Buyer'].unique()[0]
    weekday_group_buyer['Buyer_List'][i]=Buyer_List
    
weekday_group_buyer['Buyer_Count']=weekday_group_buyer['Buyer_List'].apply(lambda x: len(x))
    

In [106]:
# 3.2 Non-Buyer 

weekday_group_non_buyer=pd.DataFrame(columns={'Phone_List',"Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"},index=['1','2','3','4','5+'])
weekday_group_non_buyer=weekday_group_non_buyer.reset_index()
weekday_group_non_buyer=weekday_group_non_buyer.rename(columns={"index":"Call_Times"})
weekday_group_non_buyer=weekday_group_non_buyer[['Call_Times','Phone_List']+["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]]

for i in range(len(Non__Transacted_df_summary)):
    Phone_List=Non__Transacted_df_summary['Non_Transacted_Phones'][i]
    df_call=unique_non_transacted_phone_calls[unique_non_transacted_phone_calls['Phone_Num'].isin(Phone_List)][['Phone_Num','Start_time','Call_Seq']]
    df_call.columns=["Phone|Buyer","Start_time_Call",'Call_Seq']
    df_call['weekday']=df_call['Start_time_Call'].apply(lambda x: str(x.weekday()))
    df_call['Day_of_Week']=df_call['weekday'].apply(lambda x: x.replace("0","Monday").\
                                                                                    replace("1","Tuesday").replace("2","Wednesday").\
                                                                                   replace("3","Thursday").replace("4","Friday").\
                                                                                   replace("5","Saturday").replace("6","Sunday"))
    df_group_weekday=df_call.groupby(['Day_of_Week'])['Phone|Buyer'].count().to_frame().reset_index()
    for weekday in ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]:
        weekday_group_non_buyer[weekday][i]=df_group_weekday[df_group_weekday['Day_of_Week']==weekday]['Phone|Buyer'].unique()[0]
    weekday_group_non_buyer['Phone_List'][i]=Phone_List
    
weekday_group_non_buyer['Phone_Count']=weekday_group_non_buyer['Phone_List'].apply(lambda x: len(x))

In [107]:
weekday_group_both=weekday_group_non_buyer[['Call_Times','Phone_List']]
weekday_group_both['Buyer_List']=weekday_group_buyer['Buyer_List']

for col in ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]:
    weekday_group_both[col]=np.nan
    for i in range(len(weekday_group_both)):
        weekday_group_both[col][i]=weekday_group_buyer[col][i]+weekday_group_non_buyer[col][i]
weekday_group_both['id_count']=np.nan
for i in range(len(weekday_group_both)):
    weekday_group_both['id_count'][i]=weekday_group_buyer['Buyer_Count'][i]+weekday_group_non_buyer['Phone_Count'][i]

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/usr/local/lib/python3.6/site-

In [108]:
output_3_weekday_writer=pd.ExcelWriter(writer_folder+"Output_3_Weekday_of_Calls_"+today_str+".xlsx",engine="xlsxwriter")
output_3_weekday_overll.to_excel(output_3_weekday_writer,'weekday_overall_brief',index=False)
weekday_group_buyer.to_excel(output_3_weekday_writer,'weekday_group_buyer',index=False)
weekday_group_non_buyer.to_excel(output_3_weekday_writer,'weekday_group_non_buyer',index=False)
weekday_group_both.to_excel(output_3_weekday_writer,'weekday_group_both',index=False)
output_3_weekday_writer.save()


# Df_4 Day_Hour Call Volumes

In [109]:
hour_call_df_non_trans=unique_non_transacted_phone_calls[['Phone_Num','Start_time','Call_Seq']]
hour_call_df_non_trans.columns=["Phone|Buyer","Start_time_Call",'Call_Seq']
hour_call_df_non_trans['Hour']=hour_call_df_non_trans['Start_time_Call'].apply(lambda x: str(x.hour))
                                                                  
hour_call_df_transacted=unique_Transacted_phone_calls_matched[['defined_buyer_id','Start_time','Call_Seq']]
hour_call_df_transacted.columns=["Phone|Buyer","Start_time_Call",'Call_Seq']
hour_call_df_transacted['Hour']=hour_call_df_transacted['Start_time_Call'].apply(lambda x: str(x.hour))

hour_call_df_both=hour_call_df_non_trans.append(hour_call_df_transacted)


output_4_both=hour_call_df_both.groupby(['Hour'])['Phone|Buyer'].count().to_frame().reset_index()
output_4_both=output_4_both.rename(columns={"Phone|Buyer":"Call_From_Both"})
output_4_transacted=hour_call_df_transacted.groupby(['Hour'])['Phone|Buyer'].count().to_frame().reset_index()
output_4_transacted=output_4_transacted.rename(columns={"Phone|Buyer":"Call_From_Buyers"})
output_4_non_trans=hour_call_df_non_trans.groupby(['Hour'])['Phone|Buyer'].count().to_frame().reset_index()
output_4_non_trans=output_4_non_trans.rename(columns={"Phone|Buyer":"Call_From_Non_Buyers"})

output_4_hour_overll=pd.merge(output_4_transacted,output_4_non_trans,on=['Hour'])
output_4_hour_overll=pd.merge(output_4_hour_overll,output_4_both,on=['Hour'])
output_4_hour_overll['Hour']=output_4_hour_overll['Hour'].astype(int)
output_4_hour_overll=output_4_hour_overll.sort_values("Hour")
output_4_hour_overll=output_4_hour_overll.reset_index()
del output_4_hour_overll['index']

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [110]:
# 4.1 Buyer 
hour_group_buyer=pd.DataFrame(columns=set(['Buyer_List']+[str(x) for x in range(24)]),index=['1','2','3','4','5+'])
hour_group_buyer=hour_group_buyer.reset_index()
hour_group_buyer=hour_group_buyer.rename(columns={"index":"Call_Times"})
hour_group_buyer=hour_group_buyer[['Call_Times','Buyer_List']+[str(x) for x in range(24)]]

for i in range(len(Transaced_df_summary)):
    Buyer_List=Transaced_df_summary['buyer_list'][i]
    df_call=unique_Transacted_phone_calls_matched[unique_Transacted_phone_calls_matched['defined_buyer_id'].isin(Buyer_List)][['defined_buyer_id','Start_time','Call_Seq']]
    df_call.columns=["Phone|Buyer","Start_time_Call",'Call_Seq']
    df_call['Hour']=df_call['Start_time_Call'].apply(lambda x: str(x.hour))
    df_group_hour=df_call.groupby(['Hour'])['Phone|Buyer'].count().to_frame().reset_index()
    
    for hour in [str(x) for x in range(24)]:
        try:
            hour_group_calls=df_group_hour[df_group_hour['Hour']==hour]['Phone|Buyer'].unique()[0]
        
        except:
            hour_group_calls=0
        
        hour_group_buyer[hour][i]=hour_group_calls
    hour_group_buyer['Buyer_List'][i]=Buyer_List
    
hour_group_buyer['Buyer_Count']=hour_group_buyer['Buyer_List'].apply(lambda x: len(x))

In [111]:
# 4.2 Non-Buyer 

hour_group_non_buyer=pd.DataFrame(columns=set(['Phone_List']+[str(x) for x in range(24)]),index=['1','2','3','4','5+'])
hour_group_non_buyer=hour_group_non_buyer.reset_index()
hour_group_non_buyer=hour_group_non_buyer.rename(columns={"index":"Call_Times"})
hour_group_non_buyer=hour_group_non_buyer[['Call_Times','Phone_List']+[str(x) for x in range(24)]]

for i in range(len(Non__Transacted_df_summary)):
    Phone_List=Non__Transacted_df_summary['Non_Transacted_Phones'][i]
    df_call=unique_non_transacted_phone_calls[unique_non_transacted_phone_calls['Phone_Num'].isin(Phone_List)][['Phone_Num','Start_time','Call_Seq']]
    df_call.columns=["Phone|Buyer","Start_time_Call",'Call_Seq']
    df_call['Hour']=df_call['Start_time_Call'].apply(lambda x: str(x.hour))
    
    df_group_hour=df_call.groupby(['Hour'])['Phone|Buyer'].count().to_frame().reset_index()
    for hour in [str(x) for x in range(24)]:
        hour_group_non_buyer[hour][i]=df_group_hour[df_group_hour['Hour']==hour]['Phone|Buyer'].unique()[0]
    hour_group_non_buyer['Phone_List'][i]=Phone_List
    
hour_group_non_buyer['Phone_Count']=hour_group_non_buyer['Phone_List'].apply(lambda x: len(x))

In [112]:
hour_group_both=hour_group_non_buyer[['Call_Times','Phone_List']]
hour_group_both['Buyer_List']=hour_group_buyer['Buyer_List']

for col in [str(x) for x in range(24)]:
    hour_group_both[col]=np.nan
    for i in range(len(hour_group_both)):
        hour_group_both[col][i]=hour_group_buyer[col][i]+hour_group_non_buyer[col][i]
hour_group_both['id_count']=np.nan
for i in range(len(hour_group_both)):
    hour_group_both['id_count'][i]=hour_group_buyer['Buyer_Count'][i]+hour_group_non_buyer['Phone_Count'][i]

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/usr/local/lib/python3.6/site-

In [113]:
output_4_hour_writer=pd.ExcelWriter(writer_folder+"Output_4_hour_of_Calls_"+today_str+".xlsx",engine="xlsxwriter")
output_4_hour_overll.to_excel(output_4_hour_writer,'hour_overall_brief',index=False)
hour_group_buyer.to_excel(output_4_hour_writer,'hour_group_buyer',index=False)
hour_group_non_buyer.to_excel(output_4_hour_writer,'hour_group_non_buyer',index=False)
hour_group_both.to_excel(output_4_hour_writer,'hour_group_both',index=False)
output_4_hour_writer.save()

# Df_5 DMA Summary

In [114]:
Buyer_DMA=transaction_data_1st_Only[['defined_buyer_id','zip_cd']]
transacted_phone_DMA=unique_Transacted_phone_calls_matched[['defined_buyer_id','Source','Start_time']]
transacted_phone_DMA=pd.merge(transacted_phone_DMA,Buyer_DMA,on="defined_buyer_id",how="left")
transacted_phone_DMA=pd.merge(transacted_phone_DMA,zip_DMA,on="zip_cd",how="left")
transacted_phone_DMA['DMA']=transacted_phone_DMA['DMA'].fillna("non-US")
transacted_phone_DMA_total_calls=transacted_phone_DMA.groupby(['DMA','Source'])['Start_time'].count().to_frame().reset_index()
transacted_phone_DMA_total_calls=transacted_phone_DMA_total_calls.rename(columns={"Start_time":"Total_Transacted_Call_Count"})

transacted_phone_DMA_buyer_count=transacted_phone_DMA.groupby(['DMA','Source'])['defined_buyer_id'].apply(lambda x: len(set(x))).to_frame().reset_index()
transacted_phone_DMA_buyer_count=transacted_phone_DMA_buyer_count.rename(columns={"defined_buyer_id":"Buyer_Count"})

transacted_phone_DMA_both=pd.merge(transacted_phone_DMA_total_calls,transacted_phone_DMA_buyer_count,on=["DMA",'Source'])
transacted_phone_phone=transacted_phone_DMA_both[transacted_phone_DMA_both['Source']=="phone"]
transacted_phone_online=transacted_phone_DMA_both[transacted_phone_DMA_both['Source']=="online"]
transacted_phone_DMA_agg_both=transacted_phone_DMA_both.groupby(['DMA'])['Total_Transacted_Call_Count','Buyer_Count'].sum().reset_index()



In [115]:
every_transaction_DMA=pd.merge(transaction_data_both,zip_DMA,on='zip_cd',how="left")
every_transaction_DMA['DMA']=every_transaction_DMA['DMA'].fillna("non-US")
every_transaction_DMA_both=every_transaction_DMA.groupby(['DMA'])['Order_id'].count().to_frame().reset_index()
every_transaction_DMA_both=every_transaction_DMA_both.rename(columns={"Order_id":"Every_Single_Order_Count"})

every_transaction_DMA_Phone=every_transaction_DMA[every_transaction_DMA['Source']=='phone']
every_transaction_DMA_Phone=every_transaction_DMA_Phone.groupby(['DMA'])['Order_id'].count().to_frame().reset_index()
every_transaction_DMA_Phone=every_transaction_DMA_Phone.rename(columns={"Order_id":"Every_Phone_Order_Count"})


every_transaction_DMA_Online=every_transaction_DMA[every_transaction_DMA['Source']=='online']
every_transaction_DMA_Online=every_transaction_DMA_Online.groupby(['DMA'])['Order_id'].count().to_frame().reset_index()
every_transaction_DMA_Online=every_transaction_DMA_Online.rename(columns={"Order_id":"Every_Online_Order_Count"})



In [116]:
output_5_DMA_writer=pd.ExcelWriter(writer_folder+"Output_5_DMA_of_Calls_"+today_str+".xlsx",engine="xlsxwriter")

transacted_phone_DMA_agg_both.to_excel(output_5_DMA_writer,'transacted_phone_DMA_agg_both',index=False)
transacted_phone_phone.to_excel(output_5_DMA_writer,'transacted_phone_phone',index=False)
transacted_phone_online.to_excel(output_5_DMA_writer,'transacted_phone_online',index=False)

every_transaction_DMA_both.to_excel(output_5_DMA_writer,'every_transaction_DMA_both',index=False)
every_transaction_DMA_Phone.to_excel(output_5_DMA_writer,'every_transaction_DMA_Phone',index=False)
every_transaction_DMA_Online.to_excel(output_5_DMA_writer,'every_transaction_DMA_Online',index=False)

output_5_DMA_writer.save()

# Df_6 Memorial Week Comparison

In [117]:
# Memorial Week Range = (2018,5,24) (2018,6,3)
# Prior Week Range = (2018,5,10) (2018,6,20)
start_date_prior_week=datetime.date(2018,5,10)
end_date_prior_week=datetime.date(2018,5,20)

start_date_memorial_week=datetime.date(2018,5,24)
end_date_memorial_week=datetime.date(2018,6,3)

In [118]:
# To correct the error about more buyers than trans
# To coorect unique non transacted calls
def Time_Range_Transacted_Summary_Table(df_input,start_time,end_time):
    df_unique_matched_trans_calls=df_input[(df_input['Start_time']>=start_time) & (df_input['Start_time']<end_time+datetime.timedelta(days=1))]
    Transacted_Call_Freq_by_buyers=df_unique_matched_trans_calls.groupby(['defined_buyer_id'])['Start_time'].count().to_frame().reset_index()
    Transacted_Call_Freq_by_buyers=Transacted_Call_Freq_by_buyers.rename(columns={"Start_time":"Call_Times"})

    Transacted_Call_Freq_by_buyers_1_to_4=Transacted_Call_Freq_by_buyers[Transacted_Call_Freq_by_buyers['Call_Times']<=4]
    Transacted_Call_Freq_by_buyers_5_plus=Transacted_Call_Freq_by_buyers[Transacted_Call_Freq_by_buyers['Call_Times']>4]

    df_1=Transacted_Call_Freq_by_buyers_1_to_4.groupby(['Call_Times'])['defined_buyer_id'].apply(list).to_frame().reset_index()
    df_1=df_1.rename(columns={"defined_buyer_id":"buyer_list"})

    df_2=Transacted_Call_Freq_by_buyers_1_to_4.groupby(['Call_Times'])['defined_buyer_id'].count().to_frame().reset_index()
    df_2=df_2.rename(columns={"defined_buyer_id":"buyer_count"})

    df_buyer=pd.merge(df_1,df_2,on="Call_Times",how="outer")
    
    df_5_plus_app=pd.DataFrame({"Call_Times":"5+","buyer_list":[Transacted_Call_Freq_by_buyers_5_plus['defined_buyer_id'].unique().tolist()],"buyer_count":len(Transacted_Call_Freq_by_buyers_5_plus['defined_buyer_id'].unique().tolist())},index=[4])
    df_buyer=df_buyer.append(df_5_plus_app)
    
    
    # 1. Transacted
    Transaced_df_summary=df_buyer.copy()
    Transaced_df_summary['Total_Buyer']=np.nan
    Transaced_df_summary['Phone_Buyer']=np.nan
    Transaced_df_summary['Online_Buyer']=np.nan

    Transaced_df_summary['Total_Trans']=np.nan
    Transaced_df_summary['Phone_Trans']=np.nan
    Transaced_df_summary['Online_Trans']=np.nan

    Transaced_df_summary['Total_Transacted_Calls']=np.nan

    # Use 1st Time Transaction only : transaction_data_1st_Only
    # Because use the 1st time transaction, the total equals to the sum of both sources
    transaction_data_range=transaction_data[(transaction_data['Date_Time']>=start_time) & (transaction_data['Date_Time']<end_time+datetime.timedelta(days=1))]
    for i in range(5):
        buyer_list=Transaced_df_summary['buyer_list'][i]
        df_trans=transaction_data_range[transaction_data_range['defined_buyer_id'].isin(buyer_list)]
        type_of_order_order=df_trans.groupby(['Source'])['Order_id'].count().to_frame().reset_index()

        try:
            phone_trans_order=type_of_order_order[type_of_order_order['Source']=="phone"]['Order_id'].unique()[0]
        except:
            phone_trans_order=0
        try:
            online_trans_order=type_of_order_order[type_of_order_order['Source']=="online"]['Order_id'].unique()[0]
        except:
            online_trans_order=0
        both_trans_order=len(df_trans['Order_id'].unique())
        
        transaction_data_1st_Only_range=transaction_data_1st_Only[(transaction_data_1st_Only['Date_Time']>=start_time) &\
                                                                 (transaction_data_1st_Only['Date_Time']<end_time+datetime.timedelta(days=1))]
        df=transaction_data_1st_Only_range[transaction_data_1st_Only_range['defined_buyer_id'].isin(buyer_list)]
        type_of_order_buyer=df.groupby(['Source'])['defined_buyer_id'].apply(lambda x: len(x.unique())).to_frame().reset_index()
        
        try:
            phone_trans_buyer=type_of_order_buyer[type_of_order_buyer['Source']=="phone"]['defined_buyer_id'].unique()[0]
        except:
            phone_trans_buyer=0
        try:
            online_trans_buyer=type_of_order_buyer[type_of_order_buyer['Source']=="online"]['defined_buyer_id'].unique()[0]
        except:
            online_trans_buyer=0
        both_trans_buyer=len(df['Order_id'].unique())


        Transaced_df_summary['Phone_Trans'][i]=phone_trans_order
        Transaced_df_summary['Online_Trans'][i]=online_trans_order
        Transaced_df_summary['Total_Trans'][i]=both_trans_order

        Transaced_df_summary['Phone_Buyer'][i]=phone_trans_buyer
        Transaced_df_summary['Online_Buyer'][i]=online_trans_buyer
        Transaced_df_summary['Total_Buyer'][i]=both_trans_buyer

        df_total_call=df_unique_matched_trans_calls[df_unique_matched_trans_calls['defined_buyer_id'].isin(buyer_list)]
        total_calls=len(df_total_call)
        Transaced_df_summary['Total_Transacted_Calls'][i]=total_calls

    Transaced_df_summary=Transaced_df_summary[['Call_Times','buyer_count','buyer_list','Total_Transacted_Calls','Phone_Trans','Online_Trans','Total_Trans',
                                              'Phone_Buyer','Online_Buyer','Total_Buyer']]
    return Transaced_df_summary
    

In [119]:
#2. Non Buyers
def Time_Range_Non_Transacted_Summary_Table(df_input,start_time,end_time):
    unique_non_transacted_phone_calls_range=df_input[(df_input['Start_time']>=start_time) & (df_input['Start_time']<end_time+datetime.timedelta(days=1))]

    Non__Transacted_df_summary_count=unique_non_transacted_phone_calls_range.groupby(['Phone_Num'])['Start_time'].count().to_frame().reset_index()
    # exclusions_non_transactions=Non__Transacted_df_summary_count[Non__Transacted_df_summary_count['Start_time']>=100]
    # Non__Transacted_df_summary_count=Non__Transacted_df_summary_count[Non__Transacted_df_summary_count['Start_time']<100]


    Non__Transacted_df_summary_list=Non__Transacted_df_summary_count.groupby(['Start_time'])['Phone_Num'].apply(list).to_frame().reset_index()
    Non__Transacted_df_summary_list=Non__Transacted_df_summary_list.rename(columns={"Start_time":"Call_Times","Phone_Num":"Non_Transacted_Phones"})

    Non__Transacted_df_summary_list_1_4=Non__Transacted_df_summary_list[Non__Transacted_df_summary_list['Call_Times']<=4]

    Non__Transacted_df_summary_list_5_Plus_list_Phone=Non__Transacted_df_summary_count[Non__Transacted_df_summary_count['Start_time']>=5]['Phone_Num'].unique().tolist()
    Non__Transacted_df_summary=Non__Transacted_df_summary_list_1_4.append(pd.DataFrame({"Call_Times":"5+","Non_Transacted_Phones":[Non__Transacted_df_summary_list_5_Plus_list_Phone]}))
    Non__Transacted_df_summary['Non_Transacted_Calls']=np.nan

    Non__Transacted_df_summary=Non__Transacted_df_summary.reset_index()
    del Non__Transacted_df_summary['index']
    for i in range(len(Non__Transacted_df_summary)):
        phone_list=Non__Transacted_df_summary['Non_Transacted_Phones'][i]
        df=unique_non_transacted_phone_calls_range[unique_non_transacted_phone_calls_range['Phone_Num'].isin(phone_list)]
        counts=len(df)
        Non__Transacted_df_summary['Non_Transacted_Calls'][i]=counts
    Non__Transacted_df_summary['Unique_Non_Trancted_Call_Count']=Non__Transacted_df_summary['Non_Transacted_Phones'].apply(len)
    return Non__Transacted_df_summary

In [120]:
memorial_transacted=Time_Range_Transacted_Summary_Table(unique_Transacted_phone_calls_matched,start_date_memorial_week,end_date_memorial_week)
memorial_non_transacted=Time_Range_Non_Transacted_Summary_Table(unique_non_transacted_phone_calls,start_date_memorial_week,end_date_memorial_week)
memorial_week=pd.merge(memorial_transacted,memorial_non_transacted,on='Call_Times',how="left")

prior_transacted=Time_Range_Transacted_Summary_Table(unique_Transacted_phone_calls_matched,start_date_prior_week,end_date_prior_week)
prior_non_transacted=Time_Range_Non_Transacted_Summary_Table(unique_non_transacted_phone_calls,start_date_prior_week,end_date_prior_week)
prior_week=pd.merge(prior_transacted,prior_non_transacted,on='Call_Times',how="left")



/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See th

In [121]:
df_6_writer=pd.ExcelWriter(writer_folder+"Output_6_Memorial_Week_Comparison_"+today_str+".xlsx",engine='xlsxwriter')
memorial_week.to_excel(df_6_writer,"memorial_week",index=False)
prior_week.to_excel(df_6_writer,"prior_week",index=False)
df_6_writer.save()